# Building a data analysis company with Group Chat

In [1]:
## import required libraries
import os
from autogen import UserProxyAgent, AssistantAgent, GroupChat, GroupChatManager
from autogen import ConversableAgent
from dotenv import load_dotenv

load_dotenv(override=True) # load environment variables from .env file

True

In [2]:
## llm configuration
llm_config ={
                "model": "gpt-4o",
                "api_key": os.getenv("OPENAI_API_KEY")
            }

## The task!

In [3]:
task = "Analyze the daily closing prices of Apple (AAPL) stock for the past 1 month and create a brief report."

## Create the Agents

This group chat will include these agents:

- User_proxy or Admin: to allow the user to comment on the report and ask the writer to refine it.
- Planner: to determine relevant information needed to complete the task.
- Engineer: to write code using the defined plan by the planner.
- Executor: to execute the code written by the engineer.
- Writer: to write the report.

In [4]:
user_proxy = UserProxyAgent(
    name="Admin",
    system_message="Give the task, and send instructions to the writer to refine the stock analysis report.",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS"
)

In [5]:
## planner agent
planner = ConversableAgent(
    name="Planner",
    system_message="""
    Given a task, please determine what stock information is needed to complete the task. 
    Please note that the information will all be retrieved using Python code. 
    Please only suggest information that can be retrieved using Python code. 
    After each step is done by others, check the progress and instruct the remaining steps. 
    If a step fails, try to workaround.
    """,
    llm_config=llm_config,
)

In [6]:
## engineer agent
engineer = AssistantAgent(
    name="Engineer",
    llm_config=llm_config,
    description="An engineer that writes code to fetch and analyze stock data based on the plan provided by the planner.",
)

In [7]:
## executor agent
executor = ConversableAgent(
    name="Executor",
    system_message="Execute the code written by the engineer and report the stock data results.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 6,
        "work_dir": "coding",
        "use_docker": False,
    },
)

In [8]:
## writer agent
writer = ConversableAgent(
    name="Writer",
    llm_config=llm_config,
    system_message="Stock Analysis Report Writer. Please write stock analysis reports in markdown format (with relevant titles) and put the content in pseudo ```md``` code block. You take feedback from the admin and refine your report.",
    description="Writer. Write stock analysis reports based on the code execution results and take feedback from the admin to refine the report."
)